In [1]:
import pandas as pd
import yaml
import numpy as np
import json

def bit_clean( row:pd.Series):
    bits ={}
    for position in range(0,8):
        if (row[f'Bit{position}'] != ('(reserved)' or '(spare)')) & (isinstance(row[f'Bit{position}'],str)):
            if '[' in row[f'Bit{position}'] :
                field = row[f'Bit{position}'].split('[')

                field_name = field[0]
                bit_fields = field[1].strip(']')
                if ':' in bit_fields:
                    bit_fields = bit_fields.split(':')
                    msb = bit_fields[0]
                    lsb = bit_fields[1]
                else:
                    msb = bit_fields
                    lsb = None

            else:
                field_name = row[f'Bit{position}']

                msb = None
                lsb = None
            bits.update(
                {
                    field_name.capitalize() :{'position':position, 'msb':msb,'lsb':lsb, 'bit_attribute':row['Attribute'][position]} 
                }
            )
    return bits
    # {'position': 7, 'default': row['Bit7'].split('[')[0] if ('[' in row['Bit7'] ) else row['Bit7'] if ((row['Bit7'] != ('(reserved)' or '(spare)')) & isinstance(row['Bit7'],str)) else 0}
def convert_excel_to_yaml(excel_path, yaml_path):
    # Read Excel file
    df = pd.read_excel(excel_path,sheet_name='PAG0')
    # Create configuration dictionary
    config = {
        'device_address': 0xD0,
        'ivm6201': {}
    }
    
    # Process each row in the register map
    for _, row in df.iterrows():
        # Normalize register name
        if  str(row['Default']) != 'nan' and str(row['Hex']) != 'nan' :
            register_name = row['Register Name']
            
            # Create register entry
            if str(register_name) != 'nan':
                register_name = register_name.replace(' ', '_').capitalize()
                register_entry = {
                    'address': '0x'+ row['Hex'],
                    'default_hex': '0x'+row['Default'],
                    'default_bin': row['Bin - Default'],#bin(int(row['Default'], 16))[2:].zfill(8),
                    'attribute': row['Attribute'],
                    # 'bits': {
                    #     'bit7': bit_clean(7,row=row),#{'position': 7, 'default': row['Bit7'].split('[')[0] if ('[' in row['Bit7'] ) else row['Bit7'] if ((row['Bit7'] != ('(reserved)' or '(spare)')) & isinstance(row['Bit7'],str)) else 0},
                    #     'bit6': bit_clean(6,row=row),#{'position': 6, 'default': row['Bit6'] if row['Bit6'] != ('(reserved)' or '(spare)') else 0},
                    #     'bit5': bit_clean(5,row=row),#{'position': 5, 'default': row['Bit5'] if row['Bit5'] != ('(reserved)' or '(spare)') else 0},
                    #     'bit4': bit_clean(4,row=row),#{'position': 4, 'default': row['Bit4'] if row['Bit4'] != ('(reserved)' or '(spare)') else 0},
                    #     'bit3': bit_clean(3,row=row),#{'position': 3, 'default': row['Bit3'] if row['Bit3'] != ('(reserved)' or '(spare)') else 0},
                    #     'bit2': bit_clean(2,row=row),#{'position': 2, 'default': row['Bit2'] if row['Bit2'] != ('(reserved)' or '(spare)') else 0},
                    #     'bit1': bit_clean(1,row=row),#{'position': 1, 'default': row['Bit1'] if row['Bit1'] != ('(reserved)' or '(spare)') else 0},
                    #     'bit0': bit_clean(0,row=row),#{'position': 0, 'default': row['Bit0'] if row['Bit0'] != ('(reserved)' or '(spare)') else 0}
                    # }
                    'fields':bit_clean(row=row)
                }

                # Add register to configuration
                config['ivm6201'][register_name] = register_entry
            # print(register_entry)
    # Write to YAML file
    with open(yaml_path, 'w') as file:
        # json.dump(config,file)
        yaml.dump(config, file, default_flow_style=False)
    
    print(f"YAML configuration saved to {yaml_path}")

# Usage
convert_excel_to_yaml('IVM6201_rev0_regmap.xlsx', 'ivm6201_config.yaml')


YAML configuration saved to ivm6201_config.yaml


In [2]:
class IVM6201Configuration:
    def __init__(self, device_address=208):
        self.device_address = device_address

    class BUCK_setting_1:
        def __init__(self):
            self.address = 0x7E
            self.attribute = 'NNNNNNNN'
            self.default_hex = 0x35
            self.default_bin = '110101.0'

            class bck_cap_mod_sel:
                def __init__(self):
                    self._value = None
                    self.position = 2
                    self.lsb = 0
                    self.msb = 2
                    self.bit_attribute = 'N'

                def __get__(self, instance, owner):
                    """
                    Getter method to return value when accessed directly
                    """
                    return self._value

                def __set__(self, instance, value):
                    """
                    Setter method using = operator with validation
                    """
                    if 0 <= value <= 7:  # 3-bit field range
                        self._value = value
                    else:
                        raise ValueError("Value must be between 0 and 7")

    # Similar implementation for other bit fields

### Register Parse Expression

In [3]:
import re

def parse_register_operation(pattern):
    regex = r'(\b0x[\dA-Fa-f]+)(?:\[(\d+)(?::(\d+))?\])?__(\b0x[\dA-Fa-f]+|\d+)(?:\s*"([^"]*)")?'
    
    matches = re.findall(regex, pattern, re.IGNORECASE)
    
    for match in matches:
        register = match[0]
        msb = match[1] if match[1] else '0'
        lsb = match[2] if match[2] else msb
        value = match[3]
        comment = match[4] if match[4] else ""
        
        print(f"Register: {register}")
        print(f"Bit Range: MSB={msb}, LSB={lsb}")
        print(f"Value: {value}")
        print(f"Comment: {comment}\n")

# Test the function
test_strings = [
    # "1. 0xFE__0x00  \"write the comments within in the double quotes and give one space from instruction\"",
    # "2.0xFE[2:1]__0x01 \"value inside the register FE is 01 with MSB and LSB 2,1 respectively\"",
    # "3.0xFE[1:1]__0x1 \"for the single bit value\"",
    # "4.0x01[5:3]__0x03[4:0]__0xA4 \"Write the value 0xA4 in the regs 0x01 and 0x03 with mask\""
    "0x03[4:0]__0xa4"
]

for string in test_strings:
    parse_register_operation(string)


Register: 0x03
Bit Range: MSB=4, LSB=0
Value: 0
Comment: 



In [4]:
# regex = r'(0x[\dA-Fa-f]+)\[(\d+):(\d+)\]__(0x[\dA-Fa-f]+)\[(\d+):(\d+)\]__(0x[\dA-Fa-f]+|\d+)'
# regex = r'(0x[\dA-Fa-f]+)\[(\d+):(\d+)\]__(0x[\dA-Fa-f]+)\[(\d+):(\d+)\]__(0x[\dA-Fa-f]+|\d+)?'
regex = r'(0x[\dA-Fa-f]+)(?:\[(\d+)(?::(\d+))?\])?__(0x[\dA-Fa-f]+)(?:\[(\d+)(?::(\d+))?\])?(?:__(0x[\dA-Fa-f]+|\d+))?'
matches = re.findall(regex, "0x03[4:0]_0x03[4:0]__0xA4", re.IGNORECASE)
matches

[('0x03', '4', '0', '0xA4', '', '', '')]

In [5]:
import re 

def parse_register_operation(input_string: str):
    register_data__ = {}
    # Use re.findall to count occurrences
    undersores__ = len(re.findall(r'__', input_string))
    # check for the nu,mber of the register to write 
    if undersores__ == 1 :
        regex = r'(0x[\dA-Fa-f]+)(?:\[(\d+)(?::(\d+))?\])?__(0x[\dA-Fa-f]+|\d+)(?:\s*"([^"]*)")?'

        matches = re.findall(regex, input_string, re.IGNORECASE)

        for match in matches:
            # Register Address
            register = match[0]

            # Bit Range Handling
            if match[1] and match[2]:
                # Two numbers in brackets [MSB:LSB]
                msb = match[1]
                lsb = match[2]
            elif match[1]:
                # Single number in brackets
                msb = match[1]
                lsb = match[1]
            else:
                # No brackets
                msb = '0'
                lsb = '0'

            # Value
            value = match[3]

            # Comment
            comment = match[4] if len(match) > 4 else ""

            # print(f"Register: {register}")
            # print(f"Bit Range: MSB={msb}, LSB={lsb}")
            # print(f"Value: {value}")
            # print(f"Comment: {comment}\n")
            register_data__ = {
                "WRREGS":1,
                "REG":{
                "addr":register,
                "msb":msb,
                "lsb":lsb,
            },
                "value":value,
                "comment":comment,
            }
    
    # if the number of registers 2 
    elif undersores__ == 2 :
        # Comprehensive regex for complex register operations
        regex = r'(0x[\dA-Fa-f]+)(?:\[(\d+)(?::(\d+))?\])?__(0x[\dA-Fa-f]+)(?:\[(\d+)(?::(\d+))?\])?(?:__(0x[\dA-Fa-f]+|\d+))?(?:\s*"([^"]*)")?'    
        matches = re.findall(regex, input_string, re.IGNORECASE)
        # print(matches)
        for match in matches:
            # First Register Details
            register1 = match[0]
            msb1 = match[1] if match[1] else None
            lsb1 = match[2] if match[2] else msb1   
            # Second Register Details
            register2 = match[3] if match[3] else None
            msb2 = match[4] if match[4] else None
            lsb2 = match[5] if match[5] else msb2   
            # Value
            value = match[6] if match[6] else None  
            # Comment
            comment = match[7] if match[7] else ""  
            # print(f"First Register: {register1}")
            # print(f"First Register Bit Range: MSB={msb1}, LSB={lsb1}")  
            # if register2:
            #     print(f"Second Register: {register2}")
            #     print(f"Second Register Bit Range: MSB={msb2}, LSB={lsb2}") 
            # if value:
            #     print(f"Value: {value}")    
            # print(f"Comment: {comment}\n")
            register_data__ = {
                    "WRREGS":2,
                    "REG":{
                        "reg1":{
                            "addr":register1,
                            "msb":msb1,
                            "lsb":lsb1,
                        },
                        "reg2":{
                            "addr":register2,
                            "msb":msb2,
                            "lsb":lsb2,
                        }
                    },
                    "value":value,
                    "comment":comment
                }
    # print(register_data__)
    return register_data__
# Test strings
test_strings = [
    "0xFE__0x00  \"write the comments within in the double quotes and give one space from instruction\"",
    "0xFE[2:1]__0x01 \" value inside the register FE is 01 with MSB and LSB 2,1 respectively\"",
    "0xFE[1:1]__0x1 \" for the single bit value\"",
    "0x01[5:3]__0x03[4:0]__0xA4 \"Write the value 0xA4 in the regs 0x01 and 0x03 with mask\""
]


for string in test_strings:
    print(parse_register_operation(string))

{'WRREGS': 1, 'REG': {'addr': '0xFE', 'msb': '0', 'lsb': '0'}, 'value': '0x00', 'comment': 'write the comments within in the double quotes and give one space from instruction'}
{'WRREGS': 1, 'REG': {'addr': '0xFE', 'msb': '2', 'lsb': '1'}, 'value': '0x01', 'comment': ' value inside the register FE is 01 with MSB and LSB 2,1 respectively'}
{'WRREGS': 1, 'REG': {'addr': '0xFE', 'msb': '1', 'lsb': '1'}, 'value': '0x1', 'comment': ' for the single bit value'}
{'WRREGS': 2, 'REG': {'reg1': {'addr': '0x01', 'msb': '5', 'lsb': '3'}, 'reg2': {'addr': '0x03', 'msb': '4', 'lsb': '0'}}, 'value': '0xA4', 'comment': 'Write the value 0xA4 in the regs 0x01 and 0x03 with mask'}


In [6]:
import re

def parse_register_notation(notation):
    # Regex pattern to match register addresses with optional bit fields
    pattern = r'(0x[0-9A-Fa-f]+)(?:\[(\d+):(\d+)\])?'
    
    # Split notation by '__'
    parts = notation.split('__')
    
    # Validate notation structure
    if len(parts) < 2:
        return {}
    
    try:
        # Parse registers
        registers = []
        for part in parts[:-1]:
            match = re.match(pattern, part)
            if match:
                address = match.group(1)
                msb = int(match.group(2)) if match.group(2) else 7
                lsb = int(match.group(3)) if match.group(3) else 0
                
                registers.append({
                    'address': address,
                    'msb': msb,
                    'lsb': lsb
                })
            else:
                return {}
        
        # Parse final value
        value = int(parts[-1], 16)
        
        return {
            'registers': registers,
            'value': value
        }
    except (ValueError, TypeError):
        return {}

# Test cases
test_notations = [
    '0xFE__0x00',
    '0x20[1]__0x01',
    '0xFE[2:1]__0x01',
    '0xFE__0x01 "Select page 1"',
    '0xFE[1:1]__0x1',
    '0x01[5:3]__0x03[4:0]__0xA4',
    '0x01[5:3]__0x03[4:0]__0x6__0x8[1:2]__0xA4'
]

for notation in test_notations:
    result = parse_register_notation(notation)
    print(f"Notation: {notation}")
    if result:
        print("Registers:")
        for reg in result['registers']:
            print(f"  Address: {reg['address']}, MSB: {reg['msb']}, LSB: {reg['lsb']}")
        print(f"Value: 0x{result['value']:X}\n")
    else:
        print("No match found\n")


Notation: 0xFE__0x00
Registers:
  Address: 0xFE, MSB: 7, LSB: 0
Value: 0x0

Notation: 0x20[1]__0x01
Registers:
  Address: 0x20, MSB: 7, LSB: 0
Value: 0x1

Notation: 0xFE[2:1]__0x01
Registers:
  Address: 0xFE, MSB: 2, LSB: 1
Value: 0x1

Notation: 0xFE__0x01 "Select page 1"
No match found

Notation: 0xFE[1:1]__0x1
Registers:
  Address: 0xFE, MSB: 1, LSB: 1
Value: 0x1

Notation: 0x01[5:3]__0x03[4:0]__0xA4
Registers:
  Address: 0x01, MSB: 5, LSB: 3
  Address: 0x03, MSB: 4, LSB: 0
Value: 0xA4

Notation: 0x01[5:3]__0x03[4:0]__0x6__0x8[1:2]__0xA4
Registers:
  Address: 0x01, MSB: 5, LSB: 3
  Address: 0x03, MSB: 4, LSB: 0
  Address: 0x6, MSB: 7, LSB: 0
  Address: 0x8, MSB: 1, LSB: 2
Value: 0xA4



In [7]:
import re

def parse_register_notation(notation):
    # Regex pattern to match register addresses with optional bit fields
    # Added (?:\s*"[^"]*")? to ignore text in double quotes
    pattern = r'(0x[0-9A-Fa-f]+)(?:\[(\d+):(\d+)\])?(?:\s*"[^"]*")?'
    
    # Split notation by '__', but ignore text in quotes
    parts = re.split(r'__', notation)
    
    # Validate notation structure
    if len(parts) < 2:
        return {}
    
    try:
        # Parse registers
        registers = []
        for part in parts[:-1]:
            # Clean part by removing quotes
            part = re.sub(r'\s*"[^"]*"', '', part).strip()
            
            match = re.match(pattern, part)
            if match:
                address = match.group(1)
                msb = int(match.group(2)) if match.group(2) else 7
                lsb = int(match.group(3)) if match.group(3) else 0
                
                registers.append({
                    'address': address,
                    'msb': msb,
                    'lsb': lsb
                })
            else:
                return {}
        
        # Parse final value, removing any quotes
        final_part = re.sub(r'\s*"[^"]*"', '', parts[-1]).strip()
        value = int(final_part, 16)
        
        return {
            'registers': registers,
            'value': value
        }
    except (ValueError, TypeError):
        return {}

# Test cases
test_notations = [
    # '0xFE__0x00',
    # '0x20[1]__0x01',
    # '0xFE[2:1]__0x01',
    '0xFE__0x01 "Select page 1"',
    # '0xFE[1:1]__0x1 "hi" ',
    # '0x01[5:3]__0x03[4:0]__0xA4',
    # '0x01[5:3]__0x03[4:0]__0x6__0x8[1:2]__0xA4 "Hi" '
]

for notation in test_notations:
    result = parse_register_notation(notation)
    print(f"Notation: {notation}")
    if result:
        print("Registers:")
        for reg in result['registers']:
            print(f"  Address: {reg['address']}, MSB: {reg['msb']}, LSB: {reg['lsb']}")
        print(f"Value: 0x{result['value']:X}\n")
    else:
        print("No match found\n")


Notation: 0xFE__0x01 "Select page 1"
Registers:
  Address: 0xFE, MSB: 7, LSB: 0
Value: 0x1



In [8]:
import re

def test_register_notation_regex(notation):
    """
    Test function to validate register notation regex pattern
    
    Args:
        notation (str): Input notation string
    
    Returns:
        bool: True if pattern matches, False otherwise
    """
    # Regular expression pattern
    pattern = r'(0x[0-9A-Fa-f]+)(?:\[(\d+):(\d+)\])?'
    
    
    # Validate each register address in the notation
    for part in notation.split('__')[:-1]:
        match = re.match(pattern, part)
        if not match:
            return False
    
    return True

# Test the function
test_cases = [
    '0xFE__0x00 "hello" ',
    '0xFE[2:1]__0x01', 
    '0x20[1]__0x01 "Amux2_EN"',
    '0xFE[1:1]__0x1',
    '0x01[5:3]__0x03[4:0]__0xA4',
    '0x01[5:3]__0x03[4:0]__0x6__0x8[1:2]__0xA4'
]

for case in test_cases:
    result = test_register_notation_regex(case)
    print(f"Notation: {case}")
    print(f"Regex Match: {result}\n")


Notation: 0xFE__0x00 "hello" 
Regex Match: True

Notation: 0xFE[2:1]__0x01
Regex Match: True

Notation: 0x20[1]__0x01 "Amux2_EN"
Regex Match: True

Notation: 0xFE[1:1]__0x1
Regex Match: True

Notation: 0x01[5:3]__0x03[4:0]__0xA4
Regex Match: True

Notation: 0x01[5:3]__0x03[4:0]__0x6__0x8[1:2]__0xA4
Regex Match: True



In [9]:
import re

def test_register_notation_regex(notation):
    """
    Test function to validate register notation regex pattern
    
    Args:
        notation (str): Input notation string
    
    Returns:
        bool: True if pattern matches, False otherwise
    """
    # Modified regex pattern to handle single bit and multiple bit scenarios
    pattern = r'(0x[0-9A-Fa-f]+)(?:\[(\d+)(?::(\d+))?\])?'
    
    # Validate each register address in the notation
    for part in notation.split('__')[:-1]:
        match = re.match(pattern, part)
        if not match:
            return False
        
        # Additional validation for single bit scenario
        if match.group(2) and not match.group(3):
            # If only one number in square brackets, it's valid
            pass
    
    return True

# Test cases
test_cases = [
    '0xFE__0x00',
    '0xFE[2:1]__0x01', 
    '0x20[1]__0x01 "Amux2_EN"',
    '0xFE[1:1]__0x1',
    '0xFE[1]__0x01',  # New test case with single bit
    '0x01[5:3]__0x03[4:0]__0xA4',
    '0x01[5:3]__0x03[4:0]__0x6__0x8[1:2]__0xA4'
]

for case in test_cases:
    result = test_register_notation_regex(case)
    print(f"Notation: {case}")
    print(f"Regex Match: {result}\n")


Notation: 0xFE__0x00
Regex Match: True

Notation: 0xFE[2:1]__0x01
Regex Match: True

Notation: 0x20[1]__0x01 "Amux2_EN"
Regex Match: True

Notation: 0xFE[1:1]__0x1
Regex Match: True

Notation: 0xFE[1]__0x01
Regex Match: True

Notation: 0x01[5:3]__0x03[4:0]__0xA4
Regex Match: True

Notation: 0x01[5:3]__0x03[4:0]__0x6__0x8[1:2]__0xA4
Regex Match: True



### delay parse 

In [10]:
import re

def parse_wait_delay(input_string):
    # Updated regex to handle underscore and decimal values
    # Case-insensitive support for units
    regex = r'Wait__delay__(\d+(?:\.\d+)?)([mun])s?'
    
    match = re.search(regex, input_string, re.IGNORECASE)
    delay = {}
    if match:
        value = float(match.group(1))
        unit = match.group(2).lower()  # Convert to lowercase
        
        units = {
            'm': {'name': 'milliseconds', 'multiplier': 10**-3},
            'u': {'name': 'microseconds', 'multiplier': 10**-6},
            'n': {'name': 'nanoseconds', 'multiplier': 10**-9}
        }
        
        unit_info = units[unit]
        
        delay = {
            "value": value,
            "unit": unit_info['name'],
            "absValue": value * unit_info['multiplier']
        }
    
    return delay

# Test cases
test_strings = [
    "Wait__delay__0.5ms",
    "Wait__delay__250us",
    "Wait__delay__100ns",
    "Wait__delay__100nS"
]

for test_string in test_strings:
    result = parse_wait_delay(test_string)
    print(f"Input: {test_string}")
    print(f"Result: {result}\n")


Input: Wait__delay__0.5ms
Result: {'value': 0.5, 'unit': 'milliseconds', 'absValue': 0.0005}

Input: Wait__delay__250us
Result: {'value': 250.0, 'unit': 'microseconds', 'absValue': 0.00025}

Input: Wait__delay__100ns
Result: {'value': 100.0, 'unit': 'nanoseconds', 'absValue': 1.0000000000000001e-07}

Input: Wait__delay__100nS
Result: {'value': 100.0, 'unit': 'nanoseconds', 'absValue': 1.0000000000000001e-07}



### Parse Constant value 

In [13]:
import re

def parse_constant_value(input_string):
    """
    Parse a constant expression with value and unit, handling multipliers.

    Args:
        input_string (str): Input string like 'Const__Itest= 100mA "comment"'

    Returns:
        dict or None: Dictionary with parsed components, or None if parsing fails.
    """
    pattern = r"^Const__([a-zA-Z0-9]+)=\s*([-+]?\d*\.?\d+)([mupkMVAHzOhm]+)\s*(?:\"[^\"]*\" *)?$"
    match = re.match(pattern, input_string)

    if match:
        name = match.group(1)
        value = float(match.group(2))
        unit_str = match.group(3)

        multiplier_map = {
            'm': 1e-3, 'u': 1e-6, 'p': 1e-12,
            'k': 1e3, 'K': 1e3, 'M': 1e6
        }
        unit_type_map = {
            'A': 'current', 'V': 'voltage',
            'Ohm': 'resistance', 'Hz': 'frequency'
        }

        multiplier = multiplier_map.get(unit_str[0], 1.0)  # Get multiplier, default to 1
        unit = unit_str[1:] if len(unit_str) > 1 else unit_str #remove the first chararacter
        calculated_value = value * multiplier

        unit_type = unit_type_map.get(unit, 'unknown')

        return {
            name: calculated_value,
            'unit': unit_type,
            'multiplier': multiplier
        }
    else:
        return None

# Test cases
test_cases = [
    'Const__Itest= 100mA "itest current" ',
    'Const__Vbias= 3.3V',
    'Const__Rload= 1kOhm',
    'Const__freq= 10MHz',
    'Const__Ileak= 10uA',
    'Const__Cap= 10pF',
    'Const__Res= 10Ohm',
    'Const__Current = 5.5 mA',
    'Const__Voltage = 10.89V'
]

for case in test_cases:
    result = parse_constant_value(case)
    print(f"Input: {case}")
    print(f"Output: {result}\n")


Input: Const__Itest= 100mA "itest current" 
Output: {'Itest': 0.1, 'unit': 'current', 'multiplier': 0.001}

Input: Const__Vbias= 3.3V
Output: {'Vbias': 3.3, 'unit': 'voltage', 'multiplier': 1.0}

Input: Const__Rload= 1kOhm
Output: {'Rload': 1000.0, 'unit': 'resistance', 'multiplier': 1000.0}

Input: Const__freq= 10MHz
Output: {'freq': 10000000.0, 'unit': 'frequency', 'multiplier': 1000000.0}

Input: Const__Ileak= 10uA
Output: {'Ileak': 9.999999999999999e-06, 'unit': 'current', 'multiplier': 1e-06}

Input: Const__Cap= 10pF
Output: None

Input: Const__Res= 10Ohm
Output: {'Res': 10.0, 'unit': 'unknown', 'multiplier': 1.0}

Input: Const__Current = 5.5 mA
Output: None

Input: Const__Voltage = 10.89V
Output: None



In [14]:
import re

def test_wait_delay_regex(notation):
    """
    Test function to validate wait delay notation regex pattern
    
    Args:
        notation (str): Input notation string
    
    Returns:
        bool: True if pattern matches, False otherwise
    """
    # Regular expression pattern for wait delay
    pattern = r'Wait__delay__(\d+)([mun])s?'
    
    # Test cases
    test_cases = [
        "Wait__delay__250ms",
        "Wait__delay__500us",
        "Wait__delay__100ns"
    ]
    
    # Validate the notation against the regex pattern
    match = re.match(pattern, notation)
    
    return match is not None

# Test the function
test_cases = [
    "Wait__delay__250ms",
    "Wait__delay__500us",
    "Wait__delay__100ns",
    "Wait__delay__750ms",
    "Invalid__delay__250"
]

for case in test_cases:
    result = test_wait_delay_regex(case)
    print(f"Notation: {case}")
    print(f"Regex Match: {result}\n")


Notation: Wait__delay__250ms
Regex Match: True

Notation: Wait__delay__500us
Regex Match: True

Notation: Wait__delay__100ns
Regex Match: True

Notation: Wait__delay__750ms
Regex Match: True

Notation: Invalid__delay__250
Regex Match: False



### Force instruction Parse

In [82]:
import re

def parse_force_instruction(input_string):
    """
    Parses Force instruction to extract primary and secondary signals, value, and other details.

    Args:
        input_string (str): Input notation string.

    Returns:
        dict: A dictionary containing the parsed information.
    """
    # Updated regex to capture primary and secondary signals, values, and other details
    regex = r'Force__([A-Za-z0-9_(.+-?)]+)(?:__(.+?))?__(-?[\d.]+|OPEN|CLOSE)([KMmunp])?([VAHz])?(?:\s*"([^"]*)")?'

    match = re.match(regex, input_string, re.IGNORECASE) #Using match instead of findall
    if match:
        primary_signal = match.group(1)  # Primary signal
        if ('__' in primary_signal) and (signal := primary_signal.split('__') ):
            primary_signal = signal[0]
            secondary_signal_part = signal[-1]
        else:
            secondary_signal_part = match.group(2) #The part between the primary Signal and Value
        value = match.group(3)  # Value or OPEN/CLOSE
        multiplier = match.group(4) or ''  # Optional multiplier
        unit = match.group(5) or ''  # Optional unit
        comment = match.group(6) or ''  # Optional comment
        #Determine secondary signal (if applicable)
        secondary_signal = None
        if secondary_signal_part and (re.match(r'^[A-Za-z0-9_\+\-]+$', secondary_signal_part)): #Checking if the secondary Signal exist
          secondary_signal = secondary_signal_part

        # Multiplier conversion dictionary
        multipliers = {
            'K': 10**3,   # Kilo
            'M': 10**6,   # Mega
            'm': 10**-3,  # Milli
            'u': 10**-6,  # Micro
            'n': 10**-9,  # Nano
            'p': 10**-12  # Pico
        }

        # Handle OPEN/CLOSE scenarios
        if value.upper() == 'OPEN':
            absolute_value = 'OPEN'
            value = 'OPEN'
        elif value.upper() == 'CLOSE':
            absolute_value = 'CLOSE'
            value = 'CLOSE'
        else:
            # Calculate absolute value for numeric inputs
            try:
              absolute_value = float(value) * multipliers.get(multiplier, 1)
            except ValueError:
              return {} # If it cannot convert the value, its an invalid input

        force_data = {
            "primary_signal": primary_signal,
            "secondary_signal": secondary_signal, #Added
            "value": value,
            "multiplier": multiplier,
            "absValue": absolute_value,
            "unit": unit,
            "comment": comment
        }

        return force_data

    return {}  # Return an empty dictionary if the regex doesn't match


# Test strings (including new test cases)
test_strings = [
    # "Force__SDWN__1.1V",
    # "Force__SDWN__0.1mA",
    # "Force__V5VDRV__5.2V",
    # "Force__SDWN__0.1uA",
    # "Force__VBSO__19V",
    # "Force__SDWN__-1.1mA",
    # "Force__SDWN__OPEN \"Open the Pin Switch\"",
    "Force__V5_VDRV__CLOSE",
    "Force__OUTP1+__OUTP1-__-5V",
    "Force__SDWN__CD_DIAG__1.1V",
    # "Force__Sdwn__VCC1+__0.1mA"
]

for string in test_strings:
    print(f"Input: {string}")
    print(parse_force_instruction(string))
    print()


Input: Force__V5_VDRV__CLOSE
{'primary_signal': 'V5_VDRV', 'secondary_signal': None, 'value': 'CLOSE', 'multiplier': '', 'absValue': 'CLOSE', 'unit': '', 'comment': ''}

Input: Force__OUTP1+__OUTP1-__-5V
{'primary_signal': 'OUTP1+', 'secondary_signal': 'OUTP1-', 'value': '-5', 'multiplier': '', 'absValue': -5.0, 'unit': 'V', 'comment': ''}

Input: Force__SDWN__CD_DIAG__1.1V
{'primary_signal': 'SDWN', 'secondary_signal': 'CD_DIAG', 'value': '1.1', 'multiplier': '', 'absValue': 1.1, 'unit': 'V', 'comment': ''}



In [19]:
import re

def parse_measurements(input_text:str):
    pattern = r'Measure__([A-Za-z]+)__([A-Za-z0-9]+)(?:__([A-Za-z0-9]+))?'
    matches = re.findall(pattern, input_text)
    result = {}
    if  (matches and (matches := matches[-1])):
        result = {
            'unit': matches[0],
            'primary_signal': matches[1],
            'secondary_signal': matches[2] if matches[2] else None
        }
        # results.append(result)
    
    return result

# Test string
test_string = """Measure__Voltage__OutP__OutN1"""

result = parse_measurements(test_string)
print(result)


{'unit': 'Voltage', 'primary_signal': 'OutP', 'secondary_signal': 'OutN1'}


In [97]:
import re
# ([A-Za-z0-9_(.+-?)]+)(?:__(.+?))?
def parse_savemeas(text):
    pattern = r'SaveMeas__([A-Za-z]+)(?:__([A-Za-z0-9\+\-\_]+))'
    matches = re.match(pattern, text)
    if matches:
        result = {}
        unit = matches.group(1)

        signals_and_vars = {index: value for index, value in enumerate(matches.group(2).split('__'))}
        primary_signal = signals_and_vars.get(0,None) 
        secondary_signal = signals_and_vars.get(1,None) 
        save_variable = signals_and_vars.get(2,None)  # Handle missing last argument

        result = {
            'unit': unit,
            'primary_signal': primary_signal,
            'secondary_signal': secondary_signal,
            'save_variable': save_variable
        }

        return result
    else:
        return None

# Test cases
test_strings = [
    'SaveMeas__Voltage__OutP__OutN__var2',
    'SaveMeas__Voltage__VBOOST1+__V1',
    'SaveMeas__Voltage__SDWN1__var1',
    "SaveMeas__Voltage__CD_DIAG1__GND",
    'SaveMeas__Current__ENABLE__GND'  # Added test case
]

for string in test_strings:
    result = parse_savemeas(string)
    print(result)



{'unit': 'Voltage', 'primary_signal': 'OutP', 'secondary_signal': 'OutN', 'save_variable': 'var2'}
{'unit': 'Voltage', 'primary_signal': 'VBOOST1+', 'secondary_signal': 'V1', 'save_variable': None}
{'unit': 'Voltage', 'primary_signal': 'SDWN1', 'secondary_signal': 'var1', 'save_variable': None}
{'unit': 'Voltage', 'primary_signal': 'CD_DIAG1', 'secondary_signal': 'GND', 'save_variable': None}
{'unit': 'Current', 'primary_signal': 'ENABLE', 'secondary_signal': 'GND', 'save_variable': None}


In [21]:
import re

def test_save_measurement_regex(notation):
    """
    Test function to validate save measurement regex pattern
    
    Args:
        notation (str): Input notation string
    
    Returns:
        bool: True if pattern matches, False otherwise
    """
    # Regular expression pattern for save measurement
    pattern = r'SaveMeas__([A-Za-z]+)__([A-Za-z0-9]+)(?:__([A-Za-z0-9]+))?__([A-Za-z0-9]+)'
    
    # Validate the notation against the regex pattern
    match = re.match(pattern, notation)
    
    return match is not None

# Test cases
test_cases = [
    'SaveMeas__Voltage__OutP__OutN__var2',
    'SaveMeas__Voltage__SDWN1__var1'
]

for case in test_cases:
    result = test_save_measurement_regex(case)
    print(f"Notation: {case}")
    print(f"Regex Match: {result}\n")


Notation: SaveMeas__Voltage__OutP__OutN__var2
Regex Match: True

Notation: SaveMeas__Voltage__SDWN1__var1
Regex Match: True



In [22]:
import re

def parse_read_instruction(text):
    pattern = r'Read__(?:0x[0-9A-Fa-f]+(?:\[(\d+):(\d+)\])?__?)+([A-Za-z0-9]+)'
    match = re.match(pattern, text)
    
    if match:
        # Extract registers and their bit ranges
        registers_raw = text.split('__')[1:-1]
        registers = []
        
        for reg in registers_raw:
            if '[' in reg:
                address, bit_range = reg.split('[')
                msb, lsb = bit_range.strip(']').split(':')
                registers.append({
                    'address': address,
                    'msb': int(msb),
                    'lsb': int(lsb)
                })
            else:
                registers.append({
                    'address': reg,
                    'msb': 7,  # Default for 8-bit register
                    'lsb': 0   # Default for 8-bit register
                })
        
        return {
            'registers': registers,
            'save_variable': match.group(3)
        }
    
    return None

# Test cases
test_strings = [
    'Read__0x1__Var0x1',
    'Read__0x1[1:2]__Var0x1',
    'Read__0x1[7:5]__0x2[4:2]__0x03__Var',
    'Read__0x1[7:5]__0x2[4:2]__0x03__0x05__0x06[1:2]__Var'
]

for string in test_strings:
    result = parse_read_instruction(string)
    print(result)


{'registers': [{'address': '0x1', 'msb': 7, 'lsb': 0}], 'save_variable': 'Var0x1'}
{'registers': [{'address': '0x1', 'msb': 1, 'lsb': 2}], 'save_variable': 'Var0x1'}
{'registers': [{'address': '0x1', 'msb': 7, 'lsb': 5}, {'address': '0x2', 'msb': 4, 'lsb': 2}, {'address': '0x03', 'msb': 7, 'lsb': 0}], 'save_variable': 'Var'}
{'registers': [{'address': '0x1', 'msb': 7, 'lsb': 5}, {'address': '0x2', 'msb': 4, 'lsb': 2}, {'address': '0x03', 'msb': 7, 'lsb': 0}, {'address': '0x05', 'msb': 7, 'lsb': 0}, {'address': '0x06', 'msb': 1, 'lsb': 2}], 'save_variable': 'Var'}


In [23]:
import re

def parse_copy_instruction(text):
    # Regex pattern to match Copy instruction with optional bit ranges
    pattern = r'Copy__(?:0x[0-9A-Fa-f]+(?:\[(\d+):(\d+)\])?__0x[0-9A-Fa-f]+(?:\[(\d+):(\d+)\])?)'
    
    match = re.match(pattern, text)
    
    if match:
        # Extract registers and their bit ranges
        registers_raw = text.split('__')[1:]
        
        registers = []
        for reg in registers_raw:
            if '[' in reg:
                address, bit_range = reg.split('[')
                msb, lsb = map(int, bit_range.strip(']').split(':'))
                registers.append({
                    'address': address,
                    'msb': max(msb, lsb),
                    'lsb': min(msb, lsb)
                })
            else:
                registers.append({
                    'address': reg,
                    'msb': 7,
                    'lsb': 0
                })
        
        # Check bit range difference
        if len(registers) == 2:
            copy_diff = abs(registers[0]['msb'] - registers[0]['lsb'])
            paste_diff = abs(registers[1]['msb'] - registers[1]['lsb'])
            
            if copy_diff == paste_diff:
                return {
                    'copy_register': registers[0],
                    'paste_register': registers[1]
                }
            else:
                print("Register size does not match")
                return {}
    
    return {}

# Test cases
test_strings = [
    'Copy__0x01__0x02',
    'Copy__0x1[2:1]__0x02[3:2]'
]

for string in test_strings:
    result = parse_copy_instruction(string)
    print(result)


{'copy_register': {'address': '0x01', 'msb': 7, 'lsb': 0}, 'paste_register': {'address': '0x02', 'msb': 7, 'lsb': 0}}
{'copy_register': {'address': '0x1', 'msb': 2, 'lsb': 1}, 'paste_register': {'address': '0x02', 'msb': 3, 'lsb': 2}}


In [24]:
import re

def parse_save_instruction(text):
    # Regex pattern to match Save instruction with optional bit ranges
    # Breakdown:
    # - 'Save__' literal start of instruction
    # - '(?:0x[0-9A-Fa-f]+(?:\[(\d+):(\d+)\])?__?)+' matches multiple registers
    #   - 0x followed by hexadecimal address
    #   - Optional bit range in square brackets
    #   - Optional separator between registers
    # - '([A-Za-z0-9_]+)' captures the final save variable
    pattern = r'Save__(?:0x[0-9A-Fa-f]+(?:\[(\d+):(\d+)\])?__?)+([A-Za-z0-9_]+)'
    
    match = re.match(pattern, text)
    
    if match:
        # Split registers, excluding the save variable
        registers_raw = text.split('__')[1:-1]
        
        # Process each register
        registers = []
        for reg in registers_raw:
            if '[' in reg:
                # Handle registers with bit ranges
                address, bit_range = reg.split('[')
                msb, lsb = map(int, bit_range.strip(']').split(':'))
                registers.append({
                    'address': address,
                    'msb': max(msb, lsb),  # Ensure msb is always larger
                    'lsb': min(msb, lsb)
                })
            else:
                # Default 8-bit register configuration
                registers.append({
                    'address': reg,
                    'msb': 7,
                    'lsb': 0
                })
        
        return {
            'registers': registers,
            'save_variable': match.group(3)
        }
    
    return {}

# Test cases
test_strings = [
    'Save__0x01__var0x1',
    'Save__0x1[2:1]__var0x2',
    'Save__0x1[1:2]__0x2[3:4]__0x4__0x5__Varmulti'
]

for string in test_strings:
    result = parse_save_instruction(string)
    print(result)


{'registers': [{'address': '0x01', 'msb': 7, 'lsb': 0}], 'save_variable': 'var0x1'}
{'registers': [{'address': '0x1', 'msb': 2, 'lsb': 1}], 'save_variable': 'var0x2'}
{'registers': [{'address': '0x1', 'msb': 2, 'lsb': 1}, {'address': '0x2', 'msb': 4, 'lsb': 3}, {'address': '0x4', 'msb': 7, 'lsb': 0}, {'address': '0x5', 'msb': 7, 'lsb': 0}], 'save_variable': 'Varmulti'}


In [25]:
import re

def parse_restore_instruction(text):
    # Regex pattern to match Restore instruction with optional bit ranges
    # Breakdown:
    # - 'Restore__' literal start of instruction
    # - '(?:0x[0-9A-Fa-f]+(?:\[(\d+):(\d+)\])?__?)+' matches multiple registers
    #   - 0x followed by hexadecimal address
    #   - Optional bit range in square brackets
    #   - Optional separator between registers
    # - '([A-Za-z0-9_]+)' captures the final restore variable
    pattern = r'Restore__(?:0x[0-9A-Fa-f]+(?:\[(\d+):(\d+)\])?__?)+([A-Za-z0-9_]+)'
    
    match = re.match(pattern, text)
    
    if match:
        # Split registers, excluding the restore variable
        registers_raw = text.split('__')[1:-1]
        
        # Process each register
        registers = []
        for reg in registers_raw:
            if '[' in reg:
                # Handle registers with bit ranges
                address, bit_range = reg.split('[')
                msb, lsb = map(int, bit_range.strip(']').split(':'))
                registers.append({
                    'address': address,
                    'msb': max(msb, lsb),  # Ensure msb is always larger
                    'lsb': min(msb, lsb)
                })
            else:
                # Default 8-bit register configuration
                registers.append({
                    'address': reg,
                    'msb': 7,
                    'lsb': 0
                })
        
        return {
            'registers': registers,
            'restore_variable': match.group(3)
        }
    
    return {}

# Test cases
test_strings = [
    'Restore__0x01__var0x1',
    'Restore__0x1[2:1]__var0x2',
    'Restore__0x1[1:2]__0x2[3:4]__0x4__0x5__Varmulti'
]

for string in test_strings:
    result = parse_restore_instruction(string)
    print(result)


{'registers': [{'address': '0x01', 'msb': 7, 'lsb': 0}], 'restore_variable': 'var0x1'}
{'registers': [{'address': '0x1', 'msb': 2, 'lsb': 1}], 'restore_variable': 'var0x2'}
{'registers': [{'address': '0x1', 'msb': 2, 'lsb': 1}, {'address': '0x2', 'msb': 4, 'lsb': 3}, {'address': '0x4', 'msb': 7, 'lsb': 0}, {'address': '0x5', 'msb': 7, 'lsb': 0}], 'restore_variable': 'Varmulti'}


In [26]:
re.search(r'[pnumkMGT]', '2mV')

<re.Match object; span=(1, 2), match='m'>

# Force__Sweep Parsing 

In [27]:
import re

def parse_force_sweep_instruction(text):
    """
    Parse Force__Sweep instruction with comprehensive regex and multiplier handling
    
    Supports:
    - Multiple signal configurations
    - Default reference signal (GND)
    - Multiplier parsing (K, M, m, u, p)
    - Units: V, A, Hz, S
    - Optional step size and sweep time
    """
    # Comprehensive regex pattern breakdown:
    # 1. Force__Sweep__: Literal instruction start
    # 2. ([A-Za-z]+): Primary signal capture
    # 3. (?:__([A-Za-z]+))?: Optional reference signal (defaults to GND)
    # 4. Value patterns with optional multiplier prefixes
    pattern = r'Force__Sweep__([A-Za-z]+)(?:__([A-Za-z]+))?__(\d+(?:\.\d+)?[KMGTmupnk]?[VAHz])__(\d+(?:\.\d+)?[KMGTmupnk]?[VAHz])(?:__(\d+(?:\.\d+)?[KMGTmupnk]?[VAHzS]))?(?:__(\d+(?:\.\d+)?[KMGTmupnk]?[VAHzS]))?'
    
    match = re.match(pattern, text)
    
    if match:
        # Extended multiplier mapping with comprehensive prefixes
        multipliers = {
            'p': 1e-12,  # Pico
            'n': 1e-9,   # Nano
            'u': 1e-6,   # Micro
            'm': 1e-3,   # Milli
            'k': 1e3,    # Kilo
            'K': 1e3,    # Alternative Kilo
            'M': 1e6,    # Mega
            'G': 1e9,    # Giga
            'T': 1e12    # Tera
        }
        
        def parse_value_with_multiplier(value):
            """
            Parse numeric value with multiplier and unit
            
            Args:
                value (str): Numeric value with optional multiplier and unit
            
            Returns:
                dict: Parsed value details
            """
            if not value:
                return None
            
            # Advanced regex-based parsing
            numeric_match = re.match(r'(\d+(?:\.\d+)?)', value)
            multiplier_match = re.search(r'[KMGTmupnk]', value)
            unit_match = re.search(r'[VAHzS]', value)
            
            if not (numeric_match and unit_match):
                return None
            
            numeric_value = float(numeric_match.group(1))
            multiplier_prefix = multiplier_match.group(0) if multiplier_match else ''
            unit = unit_match.group(0)
            
            # Determine multiplier with fallback
            multiplier = multipliers.get(multiplier_prefix, 1)
            
            return {
                'raw_value': numeric_value,
                'multiplier': multiplier,
                'unit': unit,
                'final_value': numeric_value * multiplier,
                'multiplier_prefix': multiplier_prefix
            }
        
        # Extract and parse instruction components
        primary_signal = match.group(1)
        reference_signal = match.group(2) if match.group(2) else 'GND'
        initial_value = parse_value_with_multiplier(match.group(3))
        final_value = parse_value_with_multiplier(match.group(4))
        
        # Optional step size and sweep time parsing
        step_size = parse_value_with_multiplier(match.group(5)) if match.group(5) else None
        sweep_time = parse_value_with_multiplier(match.group(6)) if match.group(6) else None
        
        return {
            'primary_signal': primary_signal,
            'reference_signal': reference_signal,
            'initial_value': initial_value,
            'final_value': final_value,
            'step_size': step_size,
            'sweep_time': sweep_time
        }
    
    return {}  # Return empty dict for invalid instructions

# Comprehensive test cases
test_strings = [
    'Force__Sweep__VBAT',
    'Force__Sweep__VBOOST__VCC2+__3V__5V__0.01V',
    # 'Force__Sweep__VBAT__2V__1V',
    # 'Force__Sweep__VBAT__SDWN__2V__1V',
    # 'Force__Sweep__VBAT__GND__2V__1V__1mV',
    # 'Force__Sweep__VBAT__GND__2V__1V__1mV__100uS',
    # 'Force__Sweep__VBAT__GND__2mV__1mV__1uV__100mS',
    # 'Force__Sweep__VBAT__GND__2V__1V__100mV__10uS',
    # 'Force__Sweep__Current__VBOOST__VCC3+__-50mA__-50mA "Itest 50mA"',
]

# Process and display results for each test case
for string in test_strings:
    result = parse_force_sweep_instruction(string)
    print(f"Input: {string}")
    print(result)
    print()



Input: Force__Sweep__VBAT
{}

Input: Force__Sweep__VBOOST__VCC2+__3V__5V__0.01V
{}



In [28]:
import re

def parse_force_sweep_instruction(text):
    """
    Parse Force__Sweep instruction with comprehensive regex and multiplier handling

    Supports:
    - Multiple signal configurations
    - Default reference signal (GND)
    - Multiplier parsing (K, M, m, u, p, n, G, T)
    - Units: V, A, Hz, S
    - Optional step size and sweep time
    """
    # Comprehensive regex pattern breakdown:
    # 1. Force__Sweep__: Literal instruction start
    # 2. ([A-Za-z]+): Primary signal capture
    # 3. (?:__([A-Za-z0-9\+]+))?: Optional reference signal (defaults to GND)
    # 4. Value patterns with optional multiplier prefixes
    pattern = r'Force__Sweep__([A-Za-z]+)(?:__([A-Za-z0-9\+]+))?__([-+]?\d+(?:\.\d+)?[KMGTmupnk]?[VAHz])__([-+]?\d+(?:\.\d+)?[KMGTmupnk]?[VAHz])(?:__([-+]?\d+(?:\.\d+)?[KMGTmupnk]?[VAHzS]))?(?:__([-+]?\d+(?:\.\d+)?[KMGTmupnk]?[VAHzS]))?'

    match = re.match(pattern, text)

    if match:
        # Extended multiplier mapping with comprehensive prefixes
        multipliers = {
            'p': 1e-12,  # Pico
            'n': 1e-9,   # Nano
            'u': 1e-6,   # Micro
            'm': 1e-3,   # Milli
            'k': 1e3,    # Kilo
            'K': 1e3,    # Alternative Kilo
            'M': 1e6,    # Mega
            'G': 1e9,    # Giga
            'T': 1e12    # Tera
        }

        def parse_value_with_multiplier(value):
            """
            Parse numeric value with multiplier and unit

            Args:
                value (str): Numeric value with optional multiplier and unit

            Returns:
                dict: Parsed value details
            """
            if not value:
                return None

            # Advanced regex-based parsing
            numeric_match = re.match(r'([-+]?\d+(?:\.\d+)?)', value)  # Allow negative numbers
            multiplier_match = re.search(r'[KMGTmupnk]', value)
            unit_match = re.search(r'[VAHzS]', value)

            if not (numeric_match and unit_match):
                return None

            numeric_value = float(numeric_match.group(1))
            multiplier_prefix = multiplier_match.group(0) if multiplier_match else ''
            unit = unit_match.group(0)

            # Determine multiplier with fallback
            multiplier = multipliers.get(multiplier_prefix, 1)

            return {
                'raw_value': numeric_value,
                'multiplier': multiplier,
                'unit': unit,
                'final_value': numeric_value * multiplier,
                'multiplier_prefix': multiplier_prefix
            }

        # Extract and parse instruction components
        primary_signal = match.group(1)
        reference_signal = match.group(2) if match.group(2) else 'GND'
        initial_value = parse_value_with_multiplier(match.group(3))
        final_value = parse_value_with_multiplier(match.group(4))

        # Optional step size and sweep time parsing
        step_size = parse_value_with_multiplier(match.group(5)) if match.group(5) else None
        sweep_time = parse_value_with_multiplier(match.group(6)) if match.group(6) else None

        return {
            'primary_signal': primary_signal,
            'secondary_signal': reference_signal,  # Changed to secondary_signal for clarity
            'initial_value': initial_value,
            'final_value': final_value,
            'step_size': step_size,
            'sweep_time': sweep_time
        }

    return {}  # Return empty dict for invalid instructions

# Comprehensive test cases
test_strings = [
    'Force__Sweep__VBAT',
    'Force__Sweep__VBOOST__VCC2+__3V__5V__0.01mV__1mS',
    # 'Force__Sweep__VBAT__2V__1V',
    # 'Force__Sweep__VBAT__SDWN__2V__1V',
    # 'Force__Sweep__VBAT__GND__2V__1V__1mV',
    # 'Force__Sweep__VBAT__GND__2V__1V__1mV__100uS',
    # 'Force__Sweep__VBAT__GND__2mV__1mV__1uV__100mS',
    # 'Force__Sweep__VBAT__GND__2V__1V__100mV__10uS',
    # 'Force__Sweep__Current__VBOOST__VCC3+__-50mA__-50mA "Itest 50mA"',
]

# Process and display results for each test case
for string in test_strings:
    result = parse_force_sweep_instruction(string)
    print(f"Input: {string}")
    print(result)
    print()


Input: Force__Sweep__VBAT
{}

Input: Force__Sweep__VBOOST__VCC2+__3V__5V__0.01mV__1mS
{'primary_signal': 'VBOOST', 'secondary_signal': 'VCC2+', 'initial_value': {'raw_value': 3.0, 'multiplier': 1, 'unit': 'V', 'final_value': 3.0, 'multiplier_prefix': ''}, 'final_value': {'raw_value': 5.0, 'multiplier': 1, 'unit': 'V', 'final_value': 5.0, 'multiplier_prefix': ''}, 'step_size': {'raw_value': 0.01, 'multiplier': 0.001, 'unit': 'V', 'final_value': 1e-05, 'multiplier_prefix': 'm'}, 'sweep_time': {'raw_value': 1.0, 'multiplier': 0.001, 'unit': 'S', 'final_value': 0.001, 'multiplier_prefix': 'm'}}



## Trigger signal Parsing

In [29]:
def parse_trigger_instruction(text):
    """
    Parse Trigger instruction with LH and HL actions
    
    Args:
        text (str): Trigger instruction string
    
    Returns:
        dict: Parsed trigger details
    """
    # Regex pattern to match Trigger instruction
    pattern = r'Trigger__([A-Z]+)(?:__(\d+))?'
    
    match = re.match(pattern, text)
    
    if match:
        action = match.group(1)
        value = int(match.group(2)) if match.group(2) else None
        
        # Define action-specific details
        trigger_map = {
            'LH': {
                'action': 'LH',
                'value': 1,
                'description': 'Output change from L to H',
                'opposite_action': 'HL',
                'opposite_value': 0
            },
            'HL': {
                'action': 'HL',
                'value': 0,
                'description': 'Output change from H to L',
                'opposite_action': 'LH',
                'opposite_value': 1
            }
        }
        
        return trigger_map.get(action, {})
    
    return {}

# Test cases
test_strings = ['Trigger__LH', 'Trigger__HL']

for string in test_strings:
    result = parse_trigger_instruction(string)
    print(f"Input: {string}")
    print(result)
    print()


Input: Trigger__LH
{'action': 'LH', 'value': 1, 'description': 'Output change from L to H', 'opposite_action': 'HL', 'opposite_value': 0}

Input: Trigger__HL
{'action': 'HL', 'value': 0, 'description': 'Output change from H to L', 'opposite_action': 'LH', 'opposite_value': 1}



In [101]:
import re

def parse_trim_instruction(input_string):
    # Comprehensive regex pattern for Trim instruction
    input_string = re.sub(r'"[^"]*"', '', input_string).strip()
    pattern = r'Trim__(?:0x[0-9A-Fa-f]+(?:\[(\d+):(\d+)\])?__?)*'
    
    match = re.match(pattern, input_string)
    
    if match:
        # Split registers
        registers_raw = input_string.split('__')[1:]
        
        registers = []
        for reg in registers_raw:
            if '[' in reg:
                # Handle registers with bit ranges
                address, bit_range = reg.split('[')
                msb, lsb = map(int, bit_range.strip(']').split(':')) if ':' in bit_range else (bit_range.strip(']'),bit_range.strip(']')) 
                registers.append({
                    'address': address,
                    'msb': max(msb, lsb),  # Ensure msb is always larger
                    'lsb': min(msb, lsb)
                })
            else:
                # Default 8-bit register configuration
                registers.append({
                    'address': reg,
                    'msb': 7,
                    'lsb': 0
                })
        
        return {
            'registers': registers
        }
    
    return {}

# Test cases
test_strings = [
    'Trim__0x1',
    'Trim__0x1[2:1]',
    'Trim__0x1[1:1]__0x2[4:2]',
    'Trim__0x1[1:2]__0x2[5:4]__0x3',
    'Trim__0xBA[2:0]__0xBB[2:1]  "Trimming for OCP_LS_CH1 to have toggling on SDO"',
    'Trim__0xBB[0]__0xBC[2:0]__0xBD[2] "Trimming for OCP_LS_CH2 to have toggling on CD_DIAG"'
]

for string in test_strings:
    result = parse_trim_instruction(string)
    print(f"Input: {string}")
    print(result)
    print()


Input: Trim__0x1
{'registers': [{'address': '0x1', 'msb': 7, 'lsb': 0}]}

Input: Trim__0x1[2:1]
{'registers': [{'address': '0x1', 'msb': 2, 'lsb': 1}]}

Input: Trim__0x1[1:1]__0x2[4:2]
{'registers': [{'address': '0x1', 'msb': 1, 'lsb': 1}, {'address': '0x2', 'msb': 4, 'lsb': 2}]}

Input: Trim__0x1[1:2]__0x2[5:4]__0x3
{'registers': [{'address': '0x1', 'msb': 2, 'lsb': 1}, {'address': '0x2', 'msb': 5, 'lsb': 4}, {'address': '0x3', 'msb': 7, 'lsb': 0}]}

Input: Trim__0xBA[2:0]__0xBB[2:1]  "Trimming for OCP_LS_CH1 to have toggling on SDO"
{'registers': [{'address': '0xBA', 'msb': 2, 'lsb': 0}, {'address': '0xBB', 'msb': 2, 'lsb': 1}]}

Input: Trim__0xBB[0]__0xBC[2:0]__0xBD[2] "Trimming for OCP_LS_CH2 to have toggling on CD_DIAG"
{'registers': [{'address': '0xBB', 'msb': '0', 'lsb': '0'}, {'address': '0xBC', 'msb': 2, 'lsb': 0}, {'address': '0xBD', 'msb': '2', 'lsb': '2'}]}



In [31]:
import re 
def extract_procedure_name(notation):
    """
    Extract procedure name from Run__ notation
    
    Args:
        notation (str): Input notation string
    
    Returns:
        str or None: Procedure name if pattern matches, None otherwise
    """
    # Regular expression pattern for Run__ procedure
    pattern = r'^Run__(.+)$'
    
    # Match the pattern
    match = re.match(pattern, notation)
    
    # Return procedure name if match found
    if match:
        return match.group(1)
    
    return None

# Test cases
test_cases = [
    'Run__startup',
    'Run__startup_5.5v',
    'Start__procedure',  # Invalid case
    'Run__'  # Invalid case
]

for case in test_cases:
    result = extract_procedure_name(case)
    print(f"Notation: {case}")
    print(f"Procedure Name: {result}\n")


Notation: Run__startup
Procedure Name: startup

Notation: Run__startup_5.5v
Procedure Name: startup_5.5v

Notation: Start__procedure
Procedure Name: None

Notation: Run__
Procedure Name: None



In [32]:
import re

def test_run_procedure_regex(notation):
    """
    Test function to validate Run__ procedure notation regex pattern
    
    Args:
        notation (str): Input notation string
    
    Returns:
        bool: True if pattern matches, False otherwise
    """
    # Regular expression pattern for Run__ procedure
    pattern = r'^Run__(.+)$'
    
    # Validate the notation against the regex pattern
    match = re.match(pattern, notation)
    
    return bool(match)

# Test cases
test_cases = [
    'Run__startup',
    'Run__startup_5.5v',
    'Start__procedure',  # Invalid case
    'Run__'  # Invalid case
]

for case in test_cases:
    result = test_run_procedure_regex(case)
    print(f"Notation: {case}")
    print(f"Regex Match: {result}\n")

Notation: Run__startup
Regex Match: True

Notation: Run__startup_5.5v
Regex Match: True

Notation: Start__procedure
Regex Match: False

Notation: Run__
Regex Match: False



In [33]:
import re

def parse_meas_match_regex(input_string):
    """
    Parse Meas__Match__ instruction.

    Args:
        input_string (str): Input string.

    Returns:
        dict or None: Parsed result or None if no match.
    """
    pattern = r'^Meas__Match__([Cc]urrent|[Vv]oltage|[Ff]requency|[Rr]esistance)__([a-zA-Z0-9]+)(?:__([a-zA-Z0-9]+))?\s*__\s*([-+]?\d*\.?\d+)([munpkMVAHzOhm]+)(?:\s*".*")?$'

    match = re.match(pattern, input_string)

    if match:
        unit = match.group(1)  # e.g., Current, Voltage
        primary_signal = match.group(2)  # e.g., SDWN, Vbat
        secondary_signal = match.group(3) if match.group(3) else 'GND'  # Default to GND
        value = float(match.group(4))
        unit_str = match.group(5)

        multiplier_map = {
            'm': 1e-3, 'u': 1e-6, 'p': 1e-12,
            'k': 1e3, 'K': 1e3, 'M': 1e6, 'n': 1e-9
        }
        unit_type_map = {
            'A': 'Current', 'V': 'Voltage',
            'Ohm': 'Resistance', 'Hz': 'Frequency'
        }

        multiplier = multiplier_map.get(unit_str[0], 1.0)  # Get multiplier, default to 1
        calculated_value = value * multiplier

        unit_name = unit_type_map.get(unit_str[-1], unit) #takes the right most character from unit_str to assign it a name
        # Verify the units match
        unit_check = (
            (unit.lower() == "current" and unit_str[-1] == "A") or
            (unit.lower() == "voltage" and unit_str[-1] == "V") or
            (unit.lower() == "resistance" and unit_str[-1] in ["m","u","p","k","M","O","h","m"]) or
            (unit.lower() == "frequency" and unit_str[-1] == "z")
        )

        if not unit_check:
            return None

        return {
            'unit': unit_name,
            'primary_signal': primary_signal,
            'secondary_signal': secondary_signal,
            'value': calculated_value
        }
    else:
        return None

# Test cases
test_cases = [
    'Meas__Match__Current__SDWN__VCC__100mA "Measure the Current from SDWN with Respect to the Ground match with Expected Value is 100mA"',
    'Meas__Match__Voltage__Vbat__4.0V',
    'Meas__Match__Frequency__CLK__10kHz',
    'Meas__Match__Resistance__Rload__1kOhm',
    'Meas__Match__resistance__Rload__1kOhm',
    'Meas__Match__voltage__Vload__1kV',
    'Meas__Match__Current__Enable__627nA',
    'Meas__Match__Voltage__Enable__0.5mV',
     'Meas__Match__Voltage__Enable__0.5mA', # Invalid
    'Meas__Match__Current__Enable__5Ohm',  #Invalid
    'Meas__Match__Frequency__Enable__5V', # Invalid
    'Meas__Match__Resistance__Enable__5Hz' # Invalid
]

for case in test_cases:
    result = parse_meas_match_regex(case)
    print(f"Input: {case}")
    print(f"Output: {result}\n")


Input: Meas__Match__Current__SDWN__VCC__100mA "Measure the Current from SDWN with Respect to the Ground match with Expected Value is 100mA"
Output: {'unit': 'Current', 'primary_signal': 'SDWN', 'secondary_signal': 'VCC', 'value': 0.1}

Input: Meas__Match__Voltage__Vbat__4.0V
Output: {'unit': 'Voltage', 'primary_signal': 'Vbat', 'secondary_signal': 'GND', 'value': 4.0}

Input: Meas__Match__Frequency__CLK__10kHz
Output: {'unit': 'Frequency', 'primary_signal': 'CLK', 'secondary_signal': 'GND', 'value': 10000.0}

Input: Meas__Match__Resistance__Rload__1kOhm
Output: {'unit': 'Resistance', 'primary_signal': 'Rload', 'secondary_signal': 'GND', 'value': 1000.0}

Input: Meas__Match__resistance__Rload__1kOhm
Output: {'unit': 'resistance', 'primary_signal': 'Rload', 'secondary_signal': 'GND', 'value': 1000.0}

Input: Meas__Match__voltage__Vload__1kV
Output: {'unit': 'Voltage', 'primary_signal': 'Vload', 'secondary_signal': 'GND', 'value': 1000.0}

Input: Meas__Match__Current__Enable__627nA
Output

In [34]:
import re
import json

def parse_sweep_trig_store(text):
    """
    Parses the Sweep__Trig__Store string to extract relevant information.
    """
    pattern = r"""
    Sweep__Trig__Store___                             # Start:  Sweep__Trig__Store___
    Sweep__Signal__([A-Za-z0-9\_\+\-]+)__                 # Sweep Signal: Capture alphanumeric + underscore
    Sweeper__Reference__([A-Za-z0-9\_\+\-]+)__           # Sweeper Reference: Capture alphanumeric + underscore
    ([-+]?\d+(?:\.\d+)?[KMGTmunp]?[VAHzOhm]?)__       # Initial Value
    ([-+]?\d+(?:\.\d+)?[KMGTmunp]?[VAHzOhm]?)__       # Final Value
    ([-+]?\d+(?:\.\d+)?[KMGTmunp]?[VAHzOhm]?)?(?:__([-+]?\d+(?:\.\d+)?[KMGTmunp]?[VAHzSOhm]?))?___       # Step Size and Sweep Time (optional)
    Trig__Signal__([A-Za-z0-9\_\+\-]+)__                # Trig Signal
    Trig__Reference__([A-Za-z0-9\_\+\-]+)__           # Trig reference: Capture alphanumeric + underscore
    TrigState__([A-Za-z0-9_]+)___                  # Trig State
    (.*)                                            # Variable (capture EVERYTHING until the end)
    """

    regex = re.compile(pattern, re.VERBOSE)
    match = regex.match(text)

    if match:
        sweep_signal = match.group(1)
        sweeper_reference = match.group(2)
        initial_value_str = match.group(3)
        final_value_str = match.group(4)
        step_size_str = match.group(5)
        sweep_time_str = match.group(6) if match.group(6) else None  # handle when sweep time is not present.
        trig_signal = match.group(7)
        trig_reference = match.group(8)
        trig_state = match.group(9)
        variable = match.group(10)

        # Helper Function to safely convert to float
        def safe_convert_to_float(input_string):
            try:
                return float(input_string)
            except (ValueError, TypeError):
                return None

        # Try to extract value and multiplier and the unit
        def extract_value_unit(value_str):
            if value_str is None:
                return (None, None, None)
            unit_match = re.search(r"[VAHzOhm]$", value_str)  # Unit at the END
            unit = unit_match.group(0) if unit_match else None
            multiplier_match = re.search(r"[KMGTmunp]", value_str)
            multiplier = multiplier_match.group(0) if multiplier_match else None
            numeric_value_match = re.search(r"[-+]?\d+(?:\.\d+)?", value_str) #Get number string

            numeric_value = safe_convert_to_float(numeric_value_match.group(0) if numeric_value_match else None)  # convert to float

            #Helper method to map a numeric number based on unit.
            multipliers = {
                'K': 10**3,   # Kilo
                'M': 10**6,   # Mega
                'G': 10**9,   # Giga
                'T': 10**12,  # Tera
                'm': 10**-3,  # Milli
                'u': 10**-6,  # Micro
                'n': 10**-9,  # Nano
                'p': 10**-12  # Pico
            }
            multiplier_value = multipliers.get(multiplier, 1) if multiplier else 1
            numeric_value = numeric_value * multiplier_value if numeric_value else None #Map it again

            return (numeric_value, unit, multiplier)


        # Extract values, units and multipliers
        initial_value, initial_unit, initial_multiplier = extract_value_unit(initial_value_str)
        final_value, final_unit, final_multiplier = extract_value_unit(final_value_str)

        step_size, step_size_unit, step_size_multiplier = extract_value_unit(step_size_str)

        sweep_time = None #set default value to None;

        if sweep_time_str:
            sweep_time, sweep_time_unit, sweep_time_multiplier = extract_value_unit(sweep_time_str)


        #To take into account to do all the extractions before doing conversions
        if (initial_unit != final_unit) and (initial_unit and final_unit):
            print(f"WARNING: Units do not match")

        # Helper Function to safely convert to float and calculate from Multiplier
        def safe_convert_to_float(input_string):
            try:
                return float(input_string)
            except (ValueError, TypeError):
                return None

        trig_value = 1 if trig_state == "LH" else 0

        result = {
            "sweep_signal": sweep_signal,
            "sweeper_reference": sweeper_reference,
            "initial_value": initial_value,
            "final_value": final_value,
            "step_size": step_size,
            "sweep_time": sweep_time,
            "unit": initial_unit,
            "trig_signal": trig_signal,
            "trig_reference": trig_reference,
            "trig_state": trig_state,
            "trig_value": trig_value,
            "variable": variable,
        }
        return result
    else:
        return None

test_strings = [
  "Sweep__Trig__Store___Sweep__Signal__Signal_1+__Sweeper__Reference__Reference1-__10V__20V__1V__1mS___Trig__Signal__Signal_A-__Trig__Reference__Reference_A+__TrigState__HL___VariableData",
  "Sweep__Trig__Store___Sweep__Signal__Signal2__Sweeper__Reference__Reference2__5mA__15mA__0.1mA___Trig__Signal__SignalB__Trig__Reference__ReferenceB__TrigState__LH___AnotherVariable",
  "Sweep__Trig__Store___Sweep__Signal__Signal2__Sweeper__Reference__Reference2__5mA__15mA___Trig__Signal__SignalB__Trig__Reference__ReferenceB__TrigState__LH___AnotherVariable",
  "Sweep__Trig__Store___Sweep__Signal__Sig_nal3+__Sweeper__Reference__Reference3__0A__0A__0A___Trig__Signal__SignalC__Trig__Reference__ReferenceC__TrigState__HL___",
  "Sweep__Trig__Store___Sweep__Signal__Signal4__Sweeper__Reference__Reference4__-2.5V__-1.5V__-0.5V__10uS___Trig__Signal__SignalD__Trig__Reference__ReferenceD__TrigState__LH___NegativeValues",
  "Sweep__Trig__Store___Sweep__Signal__Signal5__Sweeper__Reference__Reference5__3.14V__6.28V___Trig__Signal__SignalE3-__Trig__Reference__ReferenceE1+__TrigState__0___DecimalValues1_[+-]",
  "Sweep__Trig__Store___Sweep__Signal__Signal6__Sweeper__Reference__Reference6__10V__20A__1Hz___Trig__Signal__SignalF__Trig__Reference__ReferenceF__TrigState__1___AllUnits",
  "Sweep__Trig__Store___Sweep__Signal__Signal7__Sweeper__Reference__Reference7__10Ohm__20Ohm__1Ohm___Trig__Signal__SignalG__Trig__Reference__ReferenceG__TrigState__HL___OhmValues",
  "Sweep__Trig__Store___Sweep__Signal__Sig_8__Sweeper__Reference__Ref_9__10V__20V__1V___Trig__Signal__SignalH1__Trig__Reference__RefH2__TrigState__LH___NumbersAndUnderscores",
  "Sweep__Trig__Store___Sweep__Signal__Signal9__Sweeper__Reference__Reference9__0.5mV__1.5mV__0.1mV___Trig__Signal__SignalI__Trig__Reference__ReferenceI__TrigState__0___MultipliersDecimals",
  "Sweep__Trig__Store___Sweep__Signal__Signal10__Sweeper__Reference__Reference10__10GHz__20THz__1GHz___Trig__Signal__SignalJ__Trig__Reference__ReferenceJ__TrigState__1___GigaTera",
  "Sweep__Trig__Store___Sweep__Signal__Signal11__Sweeper__Reference__Reference11__10V__20V__1V___Trig__Signal__SignalK__Trig__Reference__ReferenceK__TrigState__HL___Var!@#$",
  "Sweep__Trig__Store___Sweep__Signal__Signal12__Sweeper__Reference__Reference12__10V__20V__1V___Trig__Signal__SignalL__Trig__Reference__ReferenceL__TrigState__LH___",
  "Sweep__Trig__Store___Sweep__Signal__Sweep_Sig__Sweeper__Reference__Sweep_Ref__10V__20V__1V___Trig__Signal__SignalM__Trig__Reference__ReferenceM__TrigState__0___Under__1",
  "Sweep__Trig__Store___Sweep__Signal__Signal14__Sweeper__Reference__Reference14__10V__20V__1V___Trig__Signal__SignalN__Trig__Reference__ReferenceN__TrigState__HL___VariableData"
]

for test_string in test_strings:
    print(f'Input : {test_string}')
    print(f'expected output : {parse_sweep_trig_store(test_string)}')

Input : Sweep__Trig__Store___Sweep__Signal__Signal_1+__Sweeper__Reference__Reference1-__10V__20V__1V__1mS___Trig__Signal__Signal_A-__Trig__Reference__Reference_A+__TrigState__HL___VariableData
expected output : {'sweep_signal': 'Signal_1+', 'sweeper_reference': 'Reference1-', 'initial_value': 10.0, 'final_value': 20.0, 'step_size': 1.0, 'sweep_time': 0.001, 'unit': 'V', 'trig_signal': 'Signal_A-', 'trig_reference': 'Reference_A+', 'trig_state': 'HL', 'trig_value': 0, 'variable': 'VariableData'}
Input : Sweep__Trig__Store___Sweep__Signal__Signal2__Sweeper__Reference__Reference2__5mA__15mA__0.1mA___Trig__Signal__SignalB__Trig__Reference__ReferenceB__TrigState__LH___AnotherVariable
expected output : {'sweep_signal': 'Signal2', 'sweeper_reference': 'Reference2', 'initial_value': 0.005, 'final_value': 0.015, 'step_size': 0.0001, 'sweep_time': None, 'unit': 'A', 'trig_signal': 'SignalB', 'trig_reference': 'ReferenceB', 'trig_state': 'LH', 'trig_value': 1, 'variable': 'AnotherVariable'}
Input

In [35]:
import re

def validate_quoted_string(text):
    pattern = r'"(?:[^\\"]|\\.)*"'
    return re.match(pattern, text) is not None

# Test cases
print(validate_quoted_string('"Hello World"'))  # True
print(validate_quoted_string('"Escaped \\" quote"'))  # True
print(validate_quoted_string('Not a quoted string'))  # False


True
True
False


### Formula Solution

In [36]:
import ast
import operator

def solve_formula(formula_string, variables=None):
    """
    Safely evaluates a mathematical formula string with variable substitution.

    Args:
        formula_string: The mathematical formula as a string (e.g., "a + b * 2").
        variables: A dictionary containing variable names and their values
                   (e.g., {"a": 10, "b": 5}).  If None, no variables are used.

    Returns:
        The result of the evaluated formula.
        Returns None if the formula is invalid or contains disallowed operations.

    Raises:
        TypeError: If variables is not a dictionary.
        NameError: If a variable in the formula is not found in the variables dictionary.
    """

    if variables is not None and not isinstance(variables, dict):
        raise TypeError("Variables must be a dictionary.")

    # Define allowed operators (safer than eval() directly)
    safe_operators = {
        ast.Add: operator.add,
        ast.Sub: operator.sub,
        ast.Mult: operator.mul,
        ast.Div: operator.truediv,
        ast.Pow: operator.pow,
        ast.USub: operator.neg,  # Unary minus
    }

    class SafeEvaluator(ast.NodeVisitor):
        def __init__(self, variables):
            self.variables = variables if variables is not None else {}
            self.result = None

        def visit_Num(self, node):
            return node.n

        def visit_Name(self, node):
            if node.id in self.variables:
                return self.variables[node.id]
            else:
                raise NameError(f"Variable '{node.id}' not found.")

        def visit_BinOp(self, node):
            op = safe_operators.get(type(node.op))
            if op is None:
                raise ValueError(f"Unsupported operator: {type(node.op)}")
            left = self.visit(node.left)
            right = self.visit(node.right)
            return op(left, right)

        def visit_UnaryOp(self, node):
            op = safe_operators.get(type(node.op))
            if op is None:
                raise ValueError(f"Unsupported unary operator: {type(node.op)}")
            operand = self.visit(node.operand)
            return op(operand)

        def generic_visit(self, node):
            raise ValueError(f"Unsupported node type: {type(node)}")

        def evaluate(self, expression):
          try:
            tree = ast.parse(expression)
            self.result = self.visit(tree.body[0].value)  # Assuming single expression
            return self.result
          except (ValueError, NameError) as e:
            print(f"Error evaluating expression: {e}")  # Handle the exception appropriately
            return None
          except Exception as e:
            print(f"An unexpected error occurred: {e}")
            return None



    evaluator = SafeEvaluator(variables)
    return evaluator.evaluate(formula_string)


# Example usage:
formula = "a + b * 2 - c**2"
vars = {"a": 10, "b": 5, "c": 2}
result = solve_formula(formula, vars)
print(f"Result: {result}")  # Output: Result: 16.0

formula2 = "x / y + 10"
vars2 = {"x": 20, "y": 4}
result2 = solve_formula(formula2, vars2)
print(f"Result2: {result2}") # Output: Result2: 15.0

formula3 = "-z + 5"
vars3 = {"z": 3}
result3 = solve_formula(formula3, vars3)
print(f"Result3: {result3}") # Output: Result3: 2

# Example without variables:
formula4 = "3 * 4 + 2"
result4 = solve_formula(formula4)
print(f"Result4: {result4}") # Output: Result4: None

# Example with invalid formula (unsupported operator):
formula5 = "(414.02-SLOPEREAL)/414.02"  # Floor division not supported
vars5 = {'SLOPEREAL':0.6}
result5 = solve_formula(formula5, vars5)
print(f"Result5: {result5}") # Output: Error evaluating expression: Unsupported operator: <class '_ast.FloorDiv'> \n Result5: None

# Example with missing variable:
formula6 = "(353/(1+GAINCENTRAL))-CODE30"
vars6 = {"GAINCENTRAL": 0.1, "CODE30" : .4 }
result6 = solve_formula(formula6, vars6)
print(f"Result6: {result6}") # Output: Error evaluating expression: Variable 'b' not found. \n Result6: None


Result: 16
Result2: 15.0
Result3: 2
Result4: 14
Result5: 0.9985507946476015
Result6: 320.5090909090909


In [46]:
import re

def parse_calculate_expression(input_string):
    """
    Parse Calculate__<operation>[__<calculate_variable>] notation.

    Args:
        input_string (str): Input notation string

    Returns:
        dict or None: Parsed result or None if no match
    """
    # Remove anything between double quotes
    input_string = re.sub(r'"[^"]*"', '', input_string).strip()
    print(input_string)
    # Updated regex pattern to capture the required groups
    pattern = r'^(Calculate)__([a-zA-Z0-9_]+)(?:__([a-zA-Z0-9_]+))?(?:=(.*))?$'

    # Check if the pattern matches
    match = re.match(pattern, input_string)

    if match:
        prefix = match.group(1)  # Extract "Calculate"
        operation = match.group(2)  # Extract the operation
        if (calculate_variable:= match.group(3)):
            calculate_variable = calculate_variable
        elif operation :
            if '__' in operation:
                variables = operation.split('__')
                calculate_variable =variables[1] if variables[1] else None # Extract the calculate_variable
                operation = variables[0] if variables[0] else None
        #     else:
        #         calculate_variable= None
        # else:
        #     calculate_variable= None
            
        formula = match.group(4) if len(match.groups()) >= 4 and match.group(4) is not None else None  # Extract the formula

        return {
            "procedure": prefix,
            "operation": operation,
            "calculate_variable": calculate_variable,
            "formula": formula.strip() if formula else None
        }

    return None

# Example usages
print(parse_calculate_expression("Calculate__GAINLOCAL_1__GAIN_1=V1/10"))
print(parse_calculate_expression("Calculate__GAINLOCAL__=V1/10"))
print(parse_calculate_expression("Calculate__GAINLOCAL"))
print(parse_calculate_expression('Calculate__MinError "this is"'))


Calculate__GAINLOCAL_1__GAIN_1=V1/10
{'procedure': 'Calculate', 'operation': 'GAINLOCAL_1', 'calculate_variable': 'GAIN_1', 'formula': 'V1/10'}
Calculate__GAINLOCAL__=V1/10
{'procedure': 'Calculate', 'operation': 'GAINLOCAL', 'calculate_variable': None, 'formula': 'V1/10'}
Calculate__GAINLOCAL
{'procedure': 'Calculate', 'operation': 'GAINLOCAL', 'calculate_variable': None, 'formula': None}
Calculate__MinError
{'procedure': 'Calculate', 'operation': 'MinError', 'calculate_variable': None, 'formula': None}
